In [1]:
import requests
import torch
import torch.nn as nn
import os
from torchvision import models
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tqdm import tqdm

In [ ]:
from torch.utils.data import Dataset
import requests
import torch
import torch.nn as nn
# Do install:
# conda install onnx
# conda install onnxruntime
import onnxruntime as ort
import numpy as np
import json
import io
import sys
import base64
from typing import Tuple
import pickle
import os
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset

In [ ]:
os.makedirs("out/models", exist_ok=True)

In [ ]:
model = models.resnet18(weights=None)
model.fc = nn.Linear(model.fc.weight.shape[1], 10)
torch.save(model.state_dict(), "out/models/dummy_submission.pt")

NameError: name 'models' is not defined

### Load data

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
class TaskDataset(Dataset):
    def __init__(self, transform=transform):

        self.ids = []
        self.imgs = []
        self.labels = []

        # self.transform = transform

    def __getitem__(self, index) -> Tuple[int, torch.Tensor, int]:
        id_ = self.ids[index]
        img = self.imgs[index]
        # img = self.transform(img)
        label = self.labels[index]
        return id_, img, label

    def __len__(self):
        return len(self.ids)

In [ ]:
dataset = TaskDataset()

In [ ]:
dataset: TaskDataset = torch.load("./Train.pt",transform)

In [ ]:
dataset

In [ ]:
len(dataset)

100000

In [ ]:
dataset[0]

(79431, <PIL.Image.Image image mode=RGB size=32x32>, 8)

In [ ]:
images = []
ids = []
labels = []
for id, img, label in dataset:
    img = img.convert('RGB')
    images.append(img)
    ids.append(id)
    labels.append(label)


In [ ]:
imgs_tensor = [transform(img) for img in images]

In [ ]:
class TensorDataset(Dataset):
    def __init__(self, ids, imgs_tensor, labels):

        self.ids = ids
        self.imgs = imgs_tensor
        self.labels = labels

        # self.transform = transform

    def __getitem__(self, index) -> Tuple[int, torch.Tensor, int]:
        id_ = self.ids[index]
        img = self.imgs[index]
        # img = self.transform(img)
        label = self.labels[index]
        return id_, img, label

    def __len__(self):
        return len(self.ids)

In [ ]:
dataset = TensorDataset(ids, imgs_tensor, labels)


### Training

Fast Gradient Sign Method (FGSM)-generate adversarial
examples with a single gradient step

� = � + � sgn(∇!� �, �, � )

Projected Gradient Descent (PGD) - multiple random restarts.

1. Train normal model

2. Add adversarial samples during training


 maximize the loss using � and �
 ���
+∈%
� � + �, �, �

3. Minimize loss on adversarial samples


epsilon 0.01 - 0.03


Adaptive attack with more accurate gradient

compute the gradients many times, usually 10, in the forward & backward
passes, accumulate them, here in the grad_noise, and at the end average them to get a more accurate
gradient.


more layers - better. e.g. 40 if millions of params



In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input

### Evaluation

In [ ]:
#### Tests ####
# (these are being ran on the eval endpoint for every submission)

allowed_models = {
    "resnet18": models.resnet18,
    "resnet34": models.resnet34,
    "resnet50": models.resnet50,
}
with open("out/models/dummy_submission.pt", "rb") as f:
    try:
        model: torch.nn.Module = allowed_models["resnet18"](weights=None)
        model.fc = torch.nn.Linear(model.fc.weight.shape[1], 10)
    except Exception as e:
        raise Exception(
            f"Invalid model class, {e=}, only {allowed_models.keys()} are allowed",
        )
    try:
        state_dict = torch.load(f, map_location=torch.device("cpu"))
        model.load_state_dict(state_dict, strict=True)
        model.eval()
        out = model(torch.randn(1, 3, 32, 32))
    except Exception as e:
        raise Exception(f"Invalid model, {e=}")

    assert out.shape == (1, 10), "Invalid output shape"


# Send the model to the server
response = requests.post("http://34.71.138.79:9090/robustness", files={"file": open("out/models/dummy_submission.pt", "rb")}, headers={"token": "TOKEN", "model-name": "resnet18"})

# Should be 400, the clean accuracy is too low
print(response.json())

### Training Resnet

In [ ]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split

In [ ]:
# Assume your dataset has 1000 samples
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)  # 80% for training
test_size = dataset_size - train_size  # 20% for testing

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
import torch
import torchvision.models as models


In [ ]:
model = models.resnet18(pretrained=True)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Modify the first convolutional layer to accept 32x32 images
model.conv1 = nn.Conv2d(3, 64)

In [ ]:
# Modify the final fully connected layer to match your number of classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

In [ ]:
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)
model = model.to(device)

mps


In [ ]:
print("MPS available: ", torch.backends.mps.is_available())


MPS available:  True


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# Training loop
num_epochs = 10

In [ ]:
def save_checkpoint(model, optimizer, epoch, filepath):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    torch.save(checkpoint, filepath)

In [ ]:
# Training loop
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in tqdm(train_loader):
            ids, inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
        save_checkpoint(model, optimizer, epoch, f'model_checkpoint_epoch_{epoch}.pth')


In [ ]:
# Example usage:
train_model(model, train_loader, criterion, optimizer, num_epochs=10)

  0%|          | 0/2500 [00:00<?, ?it/s]

100%|██████████| 2500/2500 [02:53<00:00, 14.41it/s]


Epoch [1/10], Loss: 1.0420


100%|██████████| 2500/2500 [03:03<00:00, 13.66it/s]


Epoch [2/10], Loss: 0.9183


...

Epoch [4/10], Loss: 0.7103


100%|██████████| 2500/2500 [03:20<00:00, 12.47it/s]


Epoch [5/10], Loss: 0.5929


  1%|          | 29/2500 [00:02<03:30, 11.74it/s]


KeyboardInterrupt: 

In [ ]:
label

9

In [ ]:
def pgd_attack(model, images, labels, eps=0.03, alpha=0.01, iters=20):
    original_images = images.clone()
    for _ in range(iters):
        # print(images.dtype)
        images.requires_grad = True
        outputs = model(images)
        model.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()
        adv_images = images + alpha * images.grad.sign()
        eta = torch.clamp(adv_images - original_images, min=-eps, max=eps)
        images = torch.clamp(original_images + eta, min=0, max=1).detach_()
    return images

In [ ]:
# Training loop
def train_adv_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in tqdm(train_loader):
            ids, inputs, labels = batch
            #print(inputs.shape)
            inputs, labels = inputs.to(device), labels.to(device)
            adversarial_img = pgd_attack(model, inputs, labels)
            adversarial_img, labels = adversarial_img.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(adversarial_img)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
        save_checkpoint(model, optimizer, epoch, f'model_checkpoint_adversarial_epoch_{epoch}.pth')


In [ ]:
train_adv_model(model, train_loader, criterion, optimizer, num_epochs=3)

 29%|██▉       | 733/2500 [18:24<44:23,  1.51s/it]  


KeyboardInterrupt: 

In [ ]:
save_checkpoint(model, optimizer, 1, f'model_checkpoint_adversarial_epoch_0.3.pth')

In [ ]:
model = model.to("cpu")

In [ ]:
#### SUBMISSION ####
torch.save(model.state_dict(), "out/models/submission_1.pt")

#### Tests ####
# (these are being ran on the eval endpoint for every submission)

allowed_models = {
    "resnet18": models.resnet18,
    "resnet34": models.resnet34,
    "resnet50": models.resnet50,
}
with open("out/models/submission_1.pt", "rb") as f:
    # try:
    #     pass
    #     #model: torch.nn.Module = allowed_models["resnet18"](weights=None)
    #     #model.fc = torch.nn.Linear(model.fc.weight.shape[1], 10)
    # except Exception as e:
    #     raise Exception(
    #         f"Invalid model class, {e=}, only {allowed_models.keys()} are allowed",
    #     )
    try:
        state_dict = torch.load(f, map_location=torch.device("cpu"))
        model.load_state_dict(state_dict, strict=True)
        model.eval()
        out = model(torch.randn(1, 3, 32, 32))
    except Exception as e:
        raise Exception(f"Invalid model, {e=}")

    assert out.shape == (1, 10), "Invalid output shape"

In [ ]:
# Send the model to the server
response = requests.post("http://34.71.138.79:9090/robustness", files={"file": open("out/models/submission_1.pt", "rb")}, headers={"token": "75184352", "model-name": "resnet18"})

# Should be 400, the clean accuracy is too low
print(response.json())

{'detail': "Invalid model, e=RuntimeError('Error(s) in loading state_dict for ResNet:\\n\\tsize mismatch for conv1.weight: copying a param with shape torch.Size([64, 3, 3, 3]) from checkpoint, the shape in current model is torch.Size([64, 3, 7, 7]).')"}


In [ ]:
num_epochs =
for epoch in range(num_epochs):
    for inputs, labels in data_loader:
        # Convert tensors to require_grad
        adversarial_inputs = inputs.clone().detach().requires_grad_(True)
        labels = labels.clone().detach()

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(adversarial_inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

print('Finished Training')

AttributeError: 'int' object has no attribute 'to'